# Challenge Data : NLP

## Algorithme

In [1]:
import pandas as pd
import numpy as np

DIRECTORY = "challenge_dataset/"

# extract data
X = pd.read_csv(DIRECTORY + 'X_train.csv', sep=';')
y = pd.read_csv(DIRECTORY + 'y_train.csv', sep=';').drop(columns=['Id'])

categories = list(y.columns)
X = X['Caption']

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

n_splits = 10 # nombre de splits pour le KFold
SVC = make_pipeline(
    CountVectorizer(),
    OneVsRestClassifier(LinearSVC())
)
cv = StratifiedKFold(n_splits, shuffle=True)
scores = []
for category in categories:
    scores.append(np.mean(cross_val_score(SVC, X, y[category], cv=cv, scoring='roc_auc')))
    print("score", category, ":", scores[-1])
print("total score", np.mean(scores))

score category_1 : 0.8040423861852435
score category_2 : 0.8420099356947081
score category_3 : 0.8736114130434783
score category_4 : 0.8001808785529716
total score 0.8299611533691005


In [5]:
X_test = pd.read_csv(DIRECTORY + 'X_test.csv', sep=';')
index = X_test['Id']
X_test = X_test['Caption']

SVC = make_pipeline(
    CountVectorizer(),
    OneVsRestClassifier(LinearSVC())
)
y_pred = pd.DataFrame(index, columns=['Id'])
for category in categories:
    SVC.fit(X, y[category])
    y_pred[category] = SVC.predict(X_test)
y_pred

,Id,category_1,category_2,category_3,category_4
0,599,0,0,0,0
1,600,0,0,0,0
2,602,0,0,1,0
3,603,0,1,0,0
4,604,1,0,1,0
...,...,...,...,...,...
152,798,0,0,1,0
153,799,0,1,1,0
154,800,1,0,0,0
155,801,0,1,0,0


In [ ]:
y_pred.to_csv("results/y_pred.csv", index=False)